# Clasificador Naïve Bayes

**Naive Bayes** es un clasificador basado en el teorema de Bayes que asume que todas las features que predicen la variable dependiente de un problema son independientes entre sí. Naïve Bayes es muy fácil de construir y muy útil para grandes datasets.

El teorema de Bayes nos presenta una forma de calcular las probabilidades a posteriori dada la probabilidad de cada feature y la probabilidad a priori tal que:

> $$\mathbf{P} \left({c \mid x} \right) = \frac{\mathbf{P} \left ({x \mid c} \right) \mathbf{P} \left({c} \right)}{\mathbf{P} \left( {x} \right)}$$

En la ecuación de arriba:

* P(c|x) es la probabilidad a posteriori de la clase c dados los atributos x
* P(c) es la probabilidad a priori de la clase **c**.
* P(x|c) es la probabilidad de nuestro predictor **x** dada la clase **c**
* P(x) es la probabilidad a priori de **x**.

**Nota : ** La asunción de independencia nunca es correcta, pero funciona bien en la práctica

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
# Cargamos unos datos de prueba para construir nuestro modelo
df = pd.read_csv('tennis.csv')
df.head(20)

,outlook,temp,humidity,windy,play
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,overcast,hot,high,weak,yes
3,rainy,mild,high,weak,yes
4,rainy,cool,normal,weak,yes
5,rainy,cool,normal,strong,no
6,overcast,cool,normal,strong,yes
7,sunny,mild,high,weak,no
8,sunny,cool,normal,weak,yes
9,rainy,mild,normal,weak,yes


In [3]:
class_name = "play"
pos_class = "yes"
neg_class = "no"

Vamos a sacar las probabilidades de clase:
* prob_pos = P(play=yes)
* prob_neg = P(play=no)

In [4]:
n_pos = len(df[df[class_name]==pos_class])
n_neg = len(df[df[class_name]==neg_class])
prob_pos = n_pos/len(df)
prob_neg = n_neg/len(df)
print("P({class_name}={pos_class}) = {prob_pos}".format(class_name=class_name, pos_class=pos_class, prob_pos=prob_pos))
print("P({class_name}={neg_class}) = {prob_neg}".format(class_name=class_name, neg_class=neg_class, prob_neg=prob_neg))

P(play=yes) = 0.6428571428571429
P(play=no) = 0.35714285714285715


Una de las partes más difíciles, vamos a crear el modelo. El modelo puede estar escrito de muchas maneras diferentes. nosotros vamos a meter todas las probabilidades condicionales en un diccionario

In [5]:

def probCondCol1Col2(columnName1,columnName2,valueColumn1,valueColumn2):
    df_value2=df[columnName1][df[columnName2]==valueColumn2]
    longitud=[]
    for x in df_value2:
        if x==valueColumn1:
            longitud.append(x)
    if valueColumn2 =='yes':
        return len(longitud)/n_pos
    elif valueColumn2 =='no':
        return len(longitud)/n_neg  
    else:
        print("Error, you have to choose between yes or no")



Dada una nueva instancia que hay que predecir

x’=(Outlook=Sunny, Temperature=Cool, Humidity=High, Wind=Strong)

Ahora vamos a sacar P(x∣c) y p(x):
* prob_x_yes = P(x’|Yes)
* prob_x_no = P(x’|No)
* prob_x = P(x’)

In [6]:
x = pd.DataFrame(data={'outlook':['sunny'],'temp':['cool'],'humidity':['high'],'windy':['strong']})
x

,outlook,temp,humidity,windy
0,sunny,cool,high,strong


In [32]:
t=[]
for column in x:
    t.append(probCondCol1Col2(column,class_name,x[column][0],pos_class))
probXYes=1
for element in t:
    probXYes *= element
probXYes=probXYes*prob_pos
p=[]
for column in x:
    p.append(probCondCol1Col2(column,class_name,x[column][0],neg_class))
probXNo=1
for element in p:
    probXNo *= element
probXNo=probXNo*prob_neg
probX=probXNo+probXYes

In [33]:
print("P(x’|Yes) = {}".format(probXYes))
print("P(x’|No) = {}".format(probXNo))
print("P(x’) = {}".format(probX))

P(x’|Yes) = 0.005291005291005291
P(x’|No) = 0.02057142857142857
P(x’) = 0.02586243386243386


Compro vocal y resuelvo:

$$\mathbf{P} \left({c \mid x} \right) = \frac{\mathbf{P} \left ({x \mid c} \right) \mathbf{P} \left({c} \right)}{\mathbf{P} \left( {x} \right)}$$

Calculo aquí:
p_pos_x = P(Yes|x’)
p_neg_x = P(No|x’)

In [29]:
p_pos_x = probXYes/probX
p_neg_x = probXNo/probX
print("P(Yes|x’) = {}".format(p_pos_x))
print("P(No|x’) = {}".format(p_neg_x))

P(Yes|x’) = 0.20458265139116202
P(No|x’) = 0.795417348608838


Dejo el tenis para otro día. Con un 80% de probabilidad que no lo disfruto.